In [168]:
from os import listdir
from os.path import isfile, join
import math

In [169]:
alpha = 0.005
lmbd_legit = 1
lmbd_spm = 100
ngrams = 3

In [170]:
wordToClass = dict()

totalMessagesCount = 0
spmMessageCount = 0


In [171]:
def read_message(path):
    result = set()
    with open(path) as f:
        subject = f.readline().split()[1:]
        f.readline()
        message = f.readline().split()
        for i in range(len(subject) + 1 - ngrams):
            result.add(' '.join(subject[i:i+ngrams]))
        for i in range(len(message) + 1 - ngrams):
            result.add(' '.join(message[i:i+ngrams]))
        # result.update(subject)
        # result.update(message)
    return result

In [172]:
def get_files_from_dir(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

In [173]:
# read_message('./data/messages/part1/121spmsg62.txt')

In [174]:
root_path = './data/messages'

read_from = range(1, 10)
# read_from = [1,2,3]
test_on = 10

for i in read_from:
    path = root_path + f'/part{i}'
    all_files = get_files_from_dir(path)

    for file in all_files:
        full_file_name = join(path, file)
        messages = read_message(full_file_name)

        totalMessagesCount += 1
        index_addition = 0
        if 'spmsg' in file:
            index_addition = 1
            spmMessageCount += 1

        for word in messages:
            if wordToClass.get(word) is None:
                wordToClass[word] = [0, 0]
            wordToClass[word][index_addition] += 1


In [175]:
k = 2
class_count = [totalMessagesCount - spmMessageCount, spmMessageCount]
lam = [lmbd_legit, lmbd_spm]

In [176]:
precalc = [0.0] * k

for i in range(k):
    if class_count[i] != 0:
        precalc[i] = math.log(1.0 * class_count[i] * lam[i] / totalMessagesCount)

for w in set(wordToClass.keys()):
    st = wordToClass[w]
    for i in range(k):
        if class_count[i] != 0:
            precalc[i] += math.log(1 - (st[i] + alpha) / (class_count[i] + 2 * alpha))

In [177]:
testing_path = root_path + f'/part{test_on}'
all_testing_files = get_files_from_dir(testing_path)
total_test_mes = 0
correct = 0

In [178]:
for test in all_testing_files:
    full_file_name = join(testing_path, test)
    ws = read_message(full_file_name)
    cr = precalc.copy()

    for w in ws:
        for i in range(k):
            if class_count[i] != 0:
                if wordToClass.get(w) is not None:
                    st = wordToClass[w]
                    pr = (st[i] + alpha) / (class_count[i] + 2 * alpha)
                    cr[i] += math.log(pr) - math.log(1 - pr)

    is_legit = False

    if cr[0] >= cr[1]:
        is_legit = True

    total_test_mes += 1

    if ('spmsg' in test) != is_legit:
        correct += 1

In [179]:
print(correct / total_test_mes)


0.908256880733945
